In [1]:
import sys
sys.path.append('..')

from selenium import webdriver
from fake_useragent import UserAgent
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import multiprocessing
import json
import time
import os
import math
import argparse
import gc
from crawler.crawler_master import CrawlerMaster
import random
import psutil

In [2]:
TIME_SLEEP       = 2
TIME_OUT         = 40
MAX_COUNT_LOOP   = 100
PUBLISHER        = 'vnexpress'
RAW_DATA_PATH    = os.path.join('../data/raw_data', PUBLISHER)
KILL_FIREFOX_CMD = "pkill firefox"

In [3]:
class CrawlerMaster():
    def __init__(self):
        self.options = self.set_option_web_driver()
        self.url     = 'https://www.foody.vn/nam-dinh'

    def set_option_web_driver(self):
        options    = Options()
        ua         = UserAgent()
        user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'#ua.random
        options.add_argument(f'user-agent={user_agent}')
        options.add_argument("window-size=1920,1080")
#         options.add_argument('--headless')
        return options

In [4]:
def random_time_sleep(level):
    if level == 0:
        return random.uniform(0.2, 0.5)
    if level == 1:
        return random.uniform(0.5, 1)
    if level >= 2:
        return random.uniform(1, 2)

class CrawlerSlave():
    def __init__(self, url, options):
        self.id       = 0
        self.comments = []
        self.driver   = webdriver.Firefox(executable_path='../web_driver/geckodriver', options=options)
        # set page load timeout
        self.driver.set_page_load_timeout(TIME_OUT)
        try:
            self.driver.get(url)
        except:
            print('Time out')
            self.driver = None
        self.log_in()
        # wait for loading page
        time.sleep(TIME_SLEEP)
        self.page_down(times=10)
        
#         if self.driver != None:
#             self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight)')
    
    def log_in(self):
        log_in_button = WebDriverWait(self.driver, 30).\
                        until(EC.presence_of_element_located((By.CLASS_NAME, "account-manage")))
        log_in_button.click()
        time.sleep(random_time_sleep(level=1))
        user_name_block = WebDriverWait(self.driver, 30).\
                          until(EC.presence_of_element_located((By.XPATH, '//*[@id="Email"]')))
        user_name_block.send_keys('langtunhi96@gmail.com')
        time.sleep(random_time_sleep(level=1))
        pass_word_block = WebDriverWait(self.driver, 30).\
                          until(EC.presence_of_element_located((By.XPATH, '//*[@id="Password"]')))
        pass_word_block.send_keys('langtunhi96')
        pass_word_block.submit()
        time.sleep(random_time_sleep(level=1))
    
    def page_down(self, times):
        body = WebDriverWait(self.driver, 30).\
               until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))
        for i in range(times):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(random_time_sleep(level=0))
    
    def load_more_result(self):
        load_more_button = WebDriverWait(self.driver, 30).\
                           until(EC.presence_of_element_located((By.CLASS_NAME, "fd-btn-more")))
        while True:
            try:
                memory_percent = psutil.virtual_memory().percent
                if memory_percent > 90:
                    print('Stop because memory > 90%')
                    break
                load_more_button.click()
                self.page_down(times=1)
            except:
                break
    
    def get
    
    def get_urls(self):
        self.urls = []
        result_block = WebDriverWait(self.driver, 30).\
                       until(EC.presence_of_element_located((By.CLASS_NAME, 'content-container')))
        result_elements = result_block.find_elements_by_class_name('content-item')
        for element in result_elements:
            url = element.find_element_by_class_name('title').find_element_by_tag_name('a').get_attribute('href')
            prefix_url = 'https://www.foody.vn'
            if prefix_url not in url:
                url = prefix_url + url
            self.urls.append(url)
    
#     def save_urls(self):

In [5]:
master = CrawlerMaster()
slave = CrawlerSlave(url=master.url, options=master.options)
slave.load_more_result()
slave.get_urls()

In [6]:
# slave.get_urls()

In [7]:
slave.urls

['https://www.foody.vn/nam-dinh/panda-chan-am-thuc-nhat',
 'https://www.foody.vn/nam-dinh/bo-bit-tet-sot-thai-to-hieu',
 'https://www.foody.vn/nam-dinh/gogi-house-nuong-han-quoc-dong-a',
 'https://www.foody.vn/nam-dinh/the-pizza-company-vong-xoay-giai-phong',
 'https://www.foody.vn/nam-dinh/thanh-nam-quan',
 'https://www.foody.vn/nam-dinh/lan-anh-bun-cha',
 'https://www.foody.vn/nam-dinh/bun-cha-nha-tho-hai-ba-trung',
 'https://www.foody.vn/nam-dinh/antonio-restaurant-am-thuc-a-au',
 'https://www.foody.vn/nam-dinh/fanxipan-studio-cafe',
 'https://www.foody.vn/nam-dinh/18-cafe',
 'https://www.foody.vn/nam-dinh/tien-thoa-mien-ngan-tiet-canh',
 'https://www.foody.vn/nam-dinh/joy-fresh-tea-coffee',
 'https://www.foody.vn/nam-dinh/bun-chan-gio-bay-cau-muoi',
 'https://www.foody.vn/nam-dinh/con-cafe',
 'https://www.foody.vn/nam-dinh/che-5k',
 'https://www.foody.vn/nam-dinh/moc-quan-lau-nuong',
 'https://www.foody.vn/nam-dinh/bun-rieu-cua-ba-yen-cua-truong',
 'https://www.foody.vn/nam-dinh/sa

In [12]:
order_options_box = slave.driver.find_element_by_class_name('list-nav')

In [18]:
order_options = slave.driver.find_element_by_partial_link_text('Đặt bàn')

NoSuchElementException: Message: Unable to locate element: Đặt bàn


In [14]:
order_options

[]